# Model Experimentation

## Imports and Setup

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Locations

In [12]:
data_dir = "../data/"
input_file = data_dir + '20200326_dataset.csv'

pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'
sentinel_dir = data_dir + 'sentinel2/'

areas = ['maicao', 'riohacha', 'uribia']

## Load Dataset

In [13]:
data = pd.read_csv(input_file).reset_index(drop=True)
print('Data dimensions: {}'.format(data.shape))
print('Class distribution:\n{}'.format(data['target'].value_counts()))
print('Class distribution (normalized):\n{}'.format(data['target'].value_counts()/len(data)))
data.head(3)

Data dimensions: (334524, 112)
Class distribution:
3    248172
2     71796
1     14556
Name: target, dtype: int64
Class distribution (normalized):
3    0.741866
2    0.214621
1    0.043513
Name: target, dtype: float64


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,savi_2020,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,target,area
0,0.15970,0.13735,0.1531,0.18700,0.2090,0.26320,0.30515,0.26965,0.3327,0.0411,...,0.187614,-0.509745,0.105128,0.239614,0.449106,-0.718433,-0.042537,0.901237,3,0
1,0.15970,0.13905,0.1454,0.17845,0.2090,0.26320,0.30515,0.26395,0.3327,0.0411,...,0.177058,-0.507485,0.105128,0.247826,0.464498,-0.716955,-0.039760,0.911490,3,0
2,0.16675,0.14875,0.1589,0.18605,0.2258,0.27945,0.32070,0.28085,0.3452,0.0416,...,0.179191,-0.524371,0.073259,0.262348,0.446475,-0.722188,-0.033995,0.875915,3,0


## Resample Dataset
Resamples 30,000 negative examples per area.

In [14]:
data_area = []
for area in data['area'].unique():
    neg_sample = data[
        (data['area'] == area) 
        & (data['target'] != 1)
    ].sample(30000, replace=False, random_state=SEED)
    data_area.append(neg_sample)

pos_samples = data[data['target'] == 1]
data_area.append(pos_samples)
data = pd.concat(data_area)

data = data.reset_index(drop=True)
print('Area distribution:\n{}'.format(data['area'].value_counts()))
print('Class distribution:\n{}'.format(data['target'].value_counts()))
print('Class distribution (normalized):\n{}'.format(data['target'].value_counts()/len(data)))
data.head(3)

Area distribution:
2    40345
1    33501
0    30710
Name: area, dtype: int64
Class distribution:
3    67639
2    22361
1    14556
Name: target, dtype: int64
Class distribution (normalized):
3    0.646916
2    0.213866
1    0.139217
Name: target, dtype: float64


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,savi_2020,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,target,area
0,0.22620,0.18300,0.1790,0.1831,0.18860,0.2509,0.2981,0.2653,0.3291,0.0431,...,0.231549,-0.594230,0.211756,0.102407,0.321400,-0.798628,-0.067515,1.196927,3,0
1,0.14955,0.12935,0.1270,0.1223,0.13935,0.2424,0.2846,0.2662,0.3236,0.0388,...,0.219911,-0.529955,0.153344,0.208832,0.384915,-0.705785,-0.057656,0.871430,3,0
2,0.16860,0.15970,0.1612,0.1616,0.18550,0.3246,0.3985,0.3127,0.4143,0.0454,...,0.205571,-0.536190,0.136678,0.237628,0.413957,-0.706110,-0.052031,0.864904,3,0


## ML Pipeline

In [15]:
from sklearn.ensemble import RandomForestClassifier

### Model Training & Evaluation

In [17]:
label = 'target'
features = [column  for column in data.columns[:-2]]

data['target'] = data["target"].replace({2:0, 3:0})
print('Class distribution:\n{}'.format(data['target'].value_counts()))

Class distribution:
0    90000
1    14556
Name: target, dtype: int64


### Leave-out-area-out Cross Validation

In [18]:
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=SEED)
model_utils.geospatialcv(data, features, label, clf);

Testing on: maicao
[[29511   489]
 [  236   474]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     30000
           1       0.49      0.67      0.57       710

    accuracy                           0.98     30710
   macro avg       0.74      0.83      0.78     30710
weighted avg       0.98      0.98      0.98     30710

Testing on: riohacha
[[29990    10]
 [ 2075  1426]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     30000
           1       0.99      0.41      0.58      3501

    accuracy                           0.94     33501
   macro avg       0.96      0.70      0.77     33501
weighted avg       0.94      0.94      0.93     33501

Testing on: uribia
[[29762   238]
 [ 8458  1887]]
              precision    recall  f1-score   support

           0       0.78      0.99      0.87     30000
           1       0.89      0.18      0.30     10345

    accuracy                  

## Train Model

In [19]:
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=SEED)
X = data[features].fillna(0)
y = data[label]
clf.fit(X, y);

## Prediction

### Load Data

In [23]:
pd.set_option('use_inf_as_na', True)
area_dict = geoutils.get_filepaths(areas, sentinel_dir, pos_mask_dir, neg_mask_dir)
data = geoutils.read_bands(area_dict, 'uribia')
print('Data dimensions: {}'.format(data.shape))
data.head(3)

100%|██████████| 5/5 [00:20<00:00,  4.17s/it]

Data dimensions: (6217512, 110)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,ndvi_2020,ndbi_2020,savi_2020,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Get Predictions

In [24]:
values = (data.sum(axis=1) > 0).astype(int) - 1
pred = clf.predict_proba(data[features].fillna(0))[:, 1] + values

### Save Predictions as TIFF

In [26]:
geoutils.save_predictions(
    pred, 
    image_src=area_dict['uribia']['images_cropped'][0], 
    output_file=data_dir+'uribia_pred2.tiff'
)